<a href="https://colab.research.google.com/github/NULabTMN/ps4-PoojaKShanbhag/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [0]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2020-04-20 13:45:58--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  --.-KB/s    in 0.1s    

2020-04-20 13:45:59 (43.6 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [0]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [0]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.8104488849639893),
 ('york', 0.776707649230957),
 ('clarence', 0.7587872743606567),
 ('prince', 0.752448558807373),
 ('warwick', 0.7475429177284241),
 ('duke', 0.7406274080276489),
 ('gloucester', 0.7185962200164795),
 ('cardinal', 0.7143577337265015),
 ('princess', 0.6929662227630615),
 ('suffolk', 0.6866199374198914)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [0]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('desdemona', 0.9587864875793457),
 ('iago', 0.9528147578239441),
 ('emilia', 0.912966251373291),
 ('proteus', 0.9042910933494568),
 ('ham', 0.9024227857589722),
 ('ha', 0.897993803024292),
 ('cressida', 0.8947058320045471),
 ('lucio', 0.8938272595405579),
 ('cassio', 0.8916572332382202),
 ('imogen', 0.8911726474761963)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [0]:
model.wv.similarity('othello', 'desdemona')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.95878655

**TODO**: Your **first task** therefore is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [0]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  v1sq = np.sqrt(np.sum(np.square(v1)))
  v2sq = np.sqrt(np.sum(np.square(v2)))
  return  np.dot(v1,v2)/(v1sq*v2sq)

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.95878655

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2020-04-20 13:46:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M   104MB/s    in 0.6s    

2020-04-20 13:46:09 (104 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2020-04-20 13:46:10--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M   106MB/s    in 0.6s    

2020-04-20 13:46:11 (106 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]

--2020-04-20 13:46:12--  http://www.ccs.neu.ed

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

> Indented block



In [0]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
# print(envec)
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [0]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  for key,val in vecDict.items():
    cosine = cosim(vec,val)
    if cosine>similarity:
      similarity = cosine
      mostSimilar = key
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.5937184138759639),
 ('matrice', 0.5088361302065516),
 ('physique', 0.4555543434796393),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-04-20 13:46:22--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  --.-KB/s    in 0.08s   

2020-04-20 13:46:22 (17.3 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [0]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [0]:
def calculateAccuracy(language, dict):
  total ,accuracy ,baseline =0,0,0
  for (eng,lang,lang_word) in links:
    if eng in envec and lang == language:
      total +=1
      if eng == lang_word:
        baseline +=1
      mostsimilar, similarity= mostSimilar(envec[eng],dict)
      if mostsimilar == lang_word:
        accuracy +=1
    
  return (accuracy/total, baseline/total)

In [0]:
# Run the faster version

# def getSquaredSumDict(lang_dict):
#   dict_new = dict()
#   for key, val in lang_dict.items():
#     dict_new[key] = np.sqrt(np.sum(np.square(val)))
#   return dict_new

# def cosim_fast(v1, v2, v1sq, v2sq):
#   ## return cosine similarity between v1 and v2
#   return  np.dot(v1,v2)/(v1sq*v2sq)


# def mostSimilar_fast(vec, sq_sum1, vecDict, dict_squared_sum):
#   mostSimilar = ''
#   similarity = 0
#   for key,val in vecDict.items():
#     sq_sum2 = dict_squared_sum[key]
#     cosine = cosim_fast(vec,val,sq_sum1,sq_sum2)
#     if cosine>similarity:
#       similarity = cosine
#       mostSimilar = key
#   return (mostSimilar, similarity)

# def calculateAccuracy_fast(language, dict, dict_squared_sum_1,dict_squared_sum_2):
#   total ,accuracy ,baseline =0,0,0
#   for (eng,lang,lang_word) in links:
#     if eng in envec and lang == language:
#       total +=1
#       if eng == lang_word:
#         baseline +=1
#       mostsimilar, similarity= mostSimilar_fast(envec[eng], dict_squared_sum_1[eng], dict,dict_squared_sum_2, )
#       if mostsimilar == lang_word:
#         accuracy +=1
#   return (accuracy/total, baseline/total)

# dict_squared_sum_en = getSquaredSumDict(envec)
# dict_squared_sum_fr = getSquaredSumDict(frvec)

# accuracy_fast ,baselineAccuracy_fast = calculateAccuracy_fast('fr',frvec,dict_squared_sum_en, dict_squared_sum_fr)
# print(accuracy_fast,baselineAccuracy_fast)

0.5359205593271862 0.6742324450298915


In [0]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy ,baselineAccuracy = calculateAccuracy('fr',frvec)
print(accuracy,baselineAccuracy)

0.5359205593271862 0.6742324450298915


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [0]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
es_accuracy, es_baseline_accuracy = calculateAccuracy('es',esvec)
print(es_accuracy, es_baseline_accuracy)

0.5432884231536926 0.5173403193612774


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".

In [0]:
# Run the faster version
# dict_squared_sum_es = getSquaredSumDict(esvec)

# es_accuracy_fast, es_baseline_accuracy_fast = calculateAccuracy_fast('es',esvec,dict_squared_sum_en, dict_squared_sum_es)
# print(es_accuracy_fast,es_baseline_accuracy_fast)

0.5432884231536926 0.5173403193612774
